In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import PIL
image = PIL.Image.open('../input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg')

In [3]:
image.size

In [4]:
# Your code here; transform the image files and then load them into Keras as tensors 
# (be sure to perform a train-val-test split)
import keras

# Instantiating a generator object and normalizing the RGB values
traingen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
testgen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
valgen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_data = traingen.flow_from_directory(
    directory='../input/chest-xray-pneumonia/chest_xray/train',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=20,
    seed=42
)

test_data = testgen.flow_from_directory(
    directory='../input/chest-xray-pneumonia/chest_xray/test',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=20,
    seed=42
)

val_data = valgen.flow_from_directory(
    directory='../input/chest-xray-pneumonia/chest_xray/val',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=20,
    seed=42
)

In [5]:
baseline = keras.models.Sequential()
baseline.add(keras.Input(shape=(150, 150, 3)))
baseline.add(keras.layers.Flatten())
baseline.add(keras.layers.Dense(64, 'tanh'))
baseline.add(keras.layers.Dense(1, 'sigmoid'))



In [6]:
baseline.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['acc'])

In [7]:
baseline_results = baseline.fit_generator(train_data,
                              steps_per_epoch=100,
                              epochs=10,
                              validation_data=test_data)

In [8]:
baseline.predict(val_data, verbose=1)

## Markdown